In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numba import jit
import time

def series(n):
    p, q = 0.9, 0.8
    x = np.empty(n, dtype=int)
    x[0] = 1  
    U = np.random.uniform(0, 1, size=n)
    for t in range(1, n):
        if x[t - 1] == 0:
            x[t] = np.random.choice(np.array([0, 1]), p = [p, 1 - p])
        else:
            x[t] = np.random.choice(np.array([0, 1]), p = [1 - q, q])
    return x

n = 100000
start1 = time.time()
x = series(n)
end1 = time.time()
print(np.mean(x == 0))
print(end1 - start1)

0.66207
2.1081888675689697


In [2]:
series_numba = jit(series)

start2 = time.time()
x1 = series_numba(n)
end2 = time.time()
print(np.mean(x1 == 0))
print(end2 - start2)

0.65975
2.7664170265197754


In [3]:
%load_ext Cython

In [4]:
%%cython

import numpy as np
import time
from numpy cimport int_t, float_t
def series_cy(int n):
    x_np = np.empty(n, dtype=int)
    cdef int_t [:] x = x_np
    cdef float p = 0.9
    cdef float q = 0.8
    cdef int t
    x[0] = 1
    for t in range(1, n):
        current_x = x[t-1]
        if x[t - 1] == 0:
            x[t] = np.random.choice(np.array([0, 1]), p = [p, 1 - p])
        else:
            x[t] = np.random.choice(np.array([0, 1]), p = [1 - q, q])
    return np.asarray(x)

n = 100000
start3 = time.time()
x3 = series_cy(n)
end3 = time.time()
print(np.mean(x3 == 0))
print(end3 - start3)

0.66092
2.106001138687134


It seems that neither method helps much with the speed (they in fact make the process slower), possibly because numba and cython do not work well with numpy functions. If we try the solutions given on the website, we could see some differences. 

In [5]:
p, q = 0.1, 0.2  # Prob of leaving low and high state respectively
def compute_series(n):
    x = np.empty(n, dtype=int)
    x[0] = 1  # Start in state 1
    U = np.random.uniform(0, 1, size=n)
    for t in range(1, n):
        current_x = x[t-1]
        if current_x == 0:
            x[t] = U[t] < p
        else:
            x[t] = U[t] > q
    return x
n = 100000
%timeit compute_series(n)
compute_series_numba = jit(compute_series)
%timeit compute_series_numba(n)

10 loops, best of 3: 95.7 ms per loop
The slowest run took 120.68 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.64 ms per loop


In [7]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [11]:
%%cython
import numpy as np
from numpy cimport int_t, float_t

def compute_series_cy(int n):
    # == Create NumPy arrays first == #
    x_np = np.empty(n, dtype=int)
    U_np = np.random.uniform(0, 1, size=n)
    # == Now create memoryviews of the arrays == #
    cdef int_t [:] x = x_np
    cdef float_t [:] U = U_np
    # == Other variable declarations == #
    cdef float p = 0.1
    cdef float q = 0.2
    cdef int t
    # == Main loop == #
    x[0] = 1
    for t in range(1, n):
        current_x = x[t-1]
        if current_x == 0:
            x[t] = U[t] < p
        else:
            x[t] = U[t] > q
    return np.asarray(x)


In [12]:
%timeit compute_series_cy(n)

100 loops, best of 3: 2.57 ms per loop
